# Experimiento de reproducción modelo paper arango-resources

Esto es una reproducción y adaptación del c+odigo encontrado en el repositorio:

https://github.com/dadangewp/ACL19-SRW

Asociado a la publicación:

***A joint learning approach with knowledge injection for zero-shot
cross-lingual hate speech detection***

In [15]:
!pip install googletrans==3.1.0a0
!pip install keras_self_attention
!pip install tensorflow-addons
!pip install scipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from googletrans import Translator
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from time import sleep
import codecs
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, Flatten, concatenate
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import tensorflow_addons as tfa
import tensorflow as tf
from keras_self_attention import SeqSelfAttention
from tqdm.notebook import tqdm
import random
import scipy

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=2,
                              inter_op_parallelism_threads=2)
from keras import backend as K
tf.compat.v1.set_random_seed(42)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

translator = Translator()

In [17]:
print(tf.__version__)
print(np.__version__)
print(scipy.__version__)

2.11.0
1.21.6
1.10.0


In [18]:
# for translation
#translator.translate(text, dest='es') spanish translation
#translator.translate(text, dest='en') english translation

# We set our Seeds
random_state = 42 # our beloved seed the answer to the universe
seed = np.random.seed(random_state) # set numpy seed
random.seed(random_state) # set python seed

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Datos

In [19]:
google_drive = True
translate = False

if google_drive:
  print("Mounting your Google Drive ...")

  from google.colab import drive
  drive.flush_and_unmount()
  drive.mount('/content/drive', force_remount=True)

Mounting your Google Drive ...
Mounted at /content/drive


In [20]:
test_file = 'tweets_test_en.csv'
train_file = 'tweets_train_en.csv'
referenced = 'referenced_tweets_data.csv'
# Arango-resources
# train_arango = 'aarango_HS.xlsx'
train_arango = 'aarango_HS_en.csv'
# hateval2019
train_hateval_es = "hateval2019/hateval2019_es_en_train.csv"
test_hateval_es = "hateval2019/hateval2019_es_en_test.csv"
train_hateval_en = "hateval2019/hateval2019_en_es_train.csv"
test_hateval_en = "hateval2019/hateval2019_en_es_test.csv"
# Hatecheck
test_hatecheck = "hatecheck_cases_final_spanish_en.csv"

if google_drive:
  path = "/content/drive/MyDrive/Paper_dataton/Datos/" # Esta es la ruta
  save_path = "/content/drive/MyDrive/Paper_dataton/models_reproduction/" # Esta es la ruta de los experimentos
else:
  path = ''

data_train = pd.read_csv(path + train_file, encoding='utf-8', index_col=0)
data_test = pd.read_csv(path + test_file, encoding='utf-8', index_col=0)
data_referenced = pd.read_csv(path + referenced, encoding='utf-8', index_col=0)
data_train.rename(columns={'Odio':'HS'}, inplace = True)
data_test.rename(columns={'Odio':'HS'}, inplace = True)
# Arango
data_arango_train = pd.read_csv(path + train_arango).dropna(subset=['Usuario'])
# preprocess arango columns
data_arango_train.rename(columns={'tweet a etiquetar':'text', '12. Estereotipo':'HS'}, inplace = True)
# Hateval
data_hateval_es_train = pd.read_csv(path + train_hateval_es)
data_hateval_es_test = pd.read_csv(path + test_hateval_es)
data_hateval_en_train = pd.read_csv(path + train_hateval_en)
data_hateval_en_test = pd.read_csv(path + test_hateval_en)
# Hatecheck
data_hatecheck = pd.read_csv(path + test_hatecheck)
#data_hatecheck = data_hatecheck.drop(columns=['ref_case_id', 	'ref_templ_id', 	'templ_id', 	'case_templ', 	'gender_male', 	'gender_female', 	'label_annotated', 	'label_annotated_maj', 	'disagreement_in_case', 	'disagreement_in_template'])
data_hatecheck['HS'] = data_hatecheck['label_gold'].apply(lambda x: 1 if x == 'hateful' else 0)
data_hatecheck.rename(columns={'test_case':'text'}, inplace = True)

# print('Amount of different Authors: ' + 
# print(data_train.author_id.value_counts())


print('Len Arango: {}\nLen hateval es: {}\nLen hateval en: {}'.format(len(data_arango_train),
                                                                      len(data_hateval_es_train) + len(data_hateval_es_test),
                                                                      len(data_hateval_en_train) + len(data_hateval_en_test)))
display(data_train.head())
display(data_arango_train.head())
display(data_hateval_es_train.head())
display(data_hateval_es_test.head())
display(data_hateval_en_train.head())
display(data_hateval_en_test.head())
display(data_hatecheck.head())

Len Arango: 9834
Len hateval es: 6100
Len hateval en: 12000


,author_id,conversation_id,text,HS,Mujeres,Comunidad LGBTQ+,Comunidades Migrantes,Pueblos Originarios,text_translated
tweet_id,,,,,,,,,
1399516036240662528,1329989512391438336,(),En una amistad o soy tipo: \r\nLo peor es que ...,0,0,0,0,0,In a friendship or am I type:\r\nThe worst thi...
1320788179721560065,1319131581949378560,(),QUIEN FUE LA MARACA CULIA TE VOY A MATAR PERRA...,1,1,0,0,0,WHO WAS THE MARACA CULIA I'M GOING TO KILL YOU...
1079889645280944129,2328230546,(),menos mal q se recibe el año con ropa blanca j...,0,0,0,0,0,Good thing you receive the year in white cloth...
1369254390134145033,819218347049029633,"(1369117564811550720,)",Cuantos INFILTRADOS extranjeros hay ahi😡 https...,1,0,0,1,0,How many foreign INFILTRATES are there😡 https:...
1533854824378290176,1229941558411243520,(),#ENCONTRADO #GUAYAQUIL\r\nFecha: 06/06/22\r\nS...,0,0,0,0,0,#FOUND #GUAYAQUIL\r\nDate: 06/06/22\r\nSector:...


,CASO,Usuario,Link,text,Contexto,1. Anonimo,2. Genero,3. Mención migración,4. Mención Venezuela,5. Mención política nacional,...,11. Insulto/Sobrenombre,HS,13. Sarcasmo/Ironía/Burla,14. Pregunta Legítima,15. Evidencia,16. Figura femenina,17. Figura masculina,Codificador,Fecha,text_translated
0,23036.0,Luna,https://twitter.com/labrigidagon/status/140128...,Eran tan pero tan feministas que invisibilizab...,NaN,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Ariel,semana 1,"They were so, so feminist that they constantly..."
1,23057.0,Karen🌼,https://twitter.com/karenseguelr/status/140270...,@Eneatipo7 @Cooperativa @karina_ol Me carga en...,NaN,1.0,2.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Ariel,semana 1,@Eneatipo7 @Cooperativa @karina_ol It loads me...
2,23058.0,Patria Y Libertad,https://twitter.com/patrianueva2020/status/140...,", ¿Sabrán las femiorcas como @karina_ol y todo...",NaN,1.0,3.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,Ariel,semana 1,Will the femiorcas like @karina_ol and all the...
3,23059.0,Nubecita.Roja,https://twitter.com/RojaNubecita/status/140247...,@AntoDelLaw @jonaovalle @CarolinaLeitao @penal...,NaN,1.0,3.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Ariel,semana 1,@AntoDelLaw @jonaovalle @CarolinaLeitao @penal...
4,23061.0,Rodrigo Catalan,https://twitter.com/rodrigo140970/status/14017...,@Ciudadanolevi1 Debajo de que piedra estaba es...,NaN,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Ariel,semana 1,@Ciudadanolevi1 Under what stone was that flag...


,id,text,HS,TR,AG,text_translated
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0,Easyjet wants to double the number of female p...
1,20002,El gobierno debe crear un control estricto de ...,1,0,0,The government must create strict immigration ...
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0,I see women destroyed by workplace and street ...
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0,"— I am respectful with others, I only remind t..."
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0,Antonio Caballero and how to be tasteless and ...


,id,text,HS,TR,AG,text_translated
0,31494,@jlmanavasques Tu eres un hijo de la gran puta,0,0,0,@jlmanavasques You are a son of a bitch
1,32461,#EnCampañaM4 Y ellos que llevan humillando al ...,0,0,0,#EnCampañaM4 And they who have been humiliatin...
2,33228,@Barbijaputa Tú sí q das vergüenza diciendo q ...,0,0,0,@Barbijaputa You are ashamed saying that ETA h...
3,31699,"Vieja de mierda, te mereces todo lo que esta m...",1,1,1,"Old shit, you deserve everything that is wrong..."
4,33806,"@KarenB612 @SebaCaro8 Puta guede ql, callate p...",1,1,1,"@KarenB612 @SebaCaro8 Whore guede ql, shut up ..."


,id,text,HS,TR,AG,text_translated
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1,0,0,"Hurra, ahorrándonos $$$ de muchas maneras @pot..."
1,202,Why would young fighting age men be the vast m...,1,0,0,¿Por qué los hombres jóvenes en edad de luchar...
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1,0,0,@KamalaHarris ¡Los ilegales tiran a sus hijos ...
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0,0,0,NY Times: Los estados 'casi todos blancos' pla...
4,205,Orban in Brussels: European leaders are ignori...,0,0,0,Orban en Bruselas: los líderes europeos están ...


,id,text,HS,TR,AG,text_translated
0,34243,"@local1025 @njdotcom @GovMurphy Oh, I could ha...",0,0,0,"@local1025 @njdotcom @GovMurphy Oh, podría hab..."
1,30593,Several of the wild fires in #california and #...,0,0,0,Varios de los incendios forestales en #califor...
2,31427,@JudicialWatch My question is how do you reset...,0,0,0,@JudicialWatch Mi pregunta es cómo se reubica ...
3,31694,"#Europe, you've got a problem! We must hurry...",1,0,0,"#Europa, tienes un problema! Debemos darnos pr..."
4,31865,This is outrageous! #StopIllegalImmigration #...,1,0,0,¡Esto es indignante! #StopIllegalImmigration #...


,mhc_case_id,functionality,text,label_gold,target_ident,HS,text_translated
0,spanish-1,derog_neg_emote_h,Odio a los negros.,hateful,black people,1,I hate blacks.
1,spanish-2,derog_neg_emote_h,Odio a los gays.,hateful,gay people,1,I hate gays.
2,spanish-3,derog_neg_emote_h,Odio a los trans.,hateful,trans people,1,I hate trans.
3,spanish-4,derog_neg_emote_h,Odio a los indígenas.,hateful,indigenous people,1,I hate the natives.
4,spanish-5,derog_neg_emote_h,Odio a los judíos.,hateful,jews,1,I hate Jews.


In [21]:
# Translation
if translate:
  # translate to english our spanish dataset
  data_hateval_es_train['text_translated'] = list(map(lambda x: x.text, translator.translate(data_hateval_es_train['text'].to_list(), dest='en')))
  data_hateval_es_train.to_csv(path + "hateval2019/hateval2019_es_en_train.csv", index=False)
  sleep(60*10)
  data_hateval_es_test['text_translated'] = list(map(lambda x: x.text, translator.translate(data_hateval_es_test['text'].to_list(), dest='en')))
  data_hateval_es_test.to_csv(path + "hateval2019/hateval2019_es_en_test.csv", index=False)
  sleep(60*10)
  ## translate to spanish our english dataset
  data_hateval_en_train['text_translated'] = list(map(lambda x: x.text, translator.translate(data_hateval_en_train['text'].to_list(), dest='es')))
  data_hateval_en_train.to_csv(path + "hateval2019/hateval2019_en_es_train.csv", index=False)
  sleep(60*10)
  data_hateval_en_test['text_translated'] = list(map(lambda x: x.text, translator.translate(data_hateval_en_test['text'].to_list(), dest='es')))
  data_hateval_en_test.to_csv(path + "hateval2019/hateval2019_en_es_test.csv", index=False)
  sleep(60*10)
  ## translate to english arango chilean dataset
  data_arango_train['text_translated'] = list(map(lambda x: x.text, translator.translate(data_arango_train['text'].to_list(), dest='en')))
  data_arango_train.to_csv(path + "aarango_HS_en.csv", index=False)
  sleep(60*10)
  ## translate to english our chilean dataset
  data_train['text_translated'] = list(map(lambda x: x.text, translator.translate(data_train['text'].to_list(), dest='en')))
  data_train.to_csv(path + "tweets_train_en.csv", index=False)
  sleep(60)
  data_test['text_translated'] = list(map(lambda x: x.text, translator.translate(data_test['text'].to_list(), dest='en')))
  data_test.to_csv(path + "tweets_test_en.csv", index=False)
  ## translate Hatecheck
  data_hatecheck['text_translated'] = list(map(lambda x: x.text, translator.translate(data_hatecheck['text'].to_list(), dest='en')))
  data_hatecheck.to_csv(path + "hatecheck_cases_final_spanish_en.csv", index=False)

In [28]:
def parse_training(fp):

    y = []
    corpus = []
    # original paper encodes ANSI but sadly its a Microsoft propietary encoding
    # so for linux or any non Microsoft Windows system we use'ISO-8859-1'
    # ANSI is a superset of ISO-8859-1 in case there's any doubts
    with codecs.open(fp, encoding="ISO-8859-1") as data_in:
        for line in data_in:
            if not line.lower().startswith("tweet index"): # discard first line if it contains metadata
                line = line.rstrip() # remove trailing whitespace
                tweet = line.split("\t")[1]
                misogyny = int(line.split("\t")[0])
                y.append(misogyny)
                corpus.append(tweet)

    return corpus, y

def parse_testing(fp):

    y = []
    corpus = []
    # original paper encodes ANSI but sadly its a Microsoft propietary encoding
    # so for linux or any non Microsoft Windows system we use'ISO-8859-1'
    # ANSI is a superset of ISO-8859-1 in case there's any doubts
    with codecs.open(fp, encoding="ISO-8859-1") as data_in:
        for line in data_in:
            if not line.lower().startswith("tweet index"): # discard first line if it contains metadata
                line = line.rstrip() # remove trailing whitespace
                tweet = line.split("\t")[1]
                misogyny = int(line.split("\t")[2])
                y.append(misogyny)
                corpus.append(tweet)

    return corpus, y

def buildEmbedding1(X):
    embeddings_index = {}
    tokenizer = Tokenizer(num_words = 15000)
    tokenizer.fit_on_texts(X)
    # original paper encodes ANSI but sadly its a Microsoft propietary encoding
    # so for linux or any non Microsoft Windows system we use'ISO-8859-1'
    # ANSI is a superset of ISO-8859-1 in case there's any doubts
    with codecs.open(path + 'wiki.multi.en.vec', encoding='ISO-8859-1') as f:
        print("read embedding...")
        for line in f:
            values = line.split(" ")
            word = values[0]
            #print(word)
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
    print("vocabulary size = " + str(len(tokenizer.word_index)))
    print("embedding size = " + str(len(embeddings_index)))
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
    unk_dict = {}
    vocab = len(tokenizer.word_index)
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        elif word in unk_dict:
            embedding_matrix[i] = unk_dict[word]
        else:
            # random init, see https://github.com/bwallace/CNN-for-text-classification/blob/master/CNN_text.py
            unk_embed = np.random.random(300) * -2 + 1
            unk_dict[word] = unk_embed
            embedding_matrix[i] = unk_dict[word]
    return vocab, embedding_matrix

def buildEmbedding2(X):
    embeddings_index = {}
    tokenizer = Tokenizer(num_words = 15000)
    tokenizer.fit_on_texts(X)
    # original paper encodes ANSI but sadly its a Microsoft propietary encoding
    # so for linux or any non Microsoft Windows system we use'ISO-8859-1'
    # ANSI is a superset of ISO-8859-1 in case there's any doubts
    with codecs.open(path + 'wiki.multi.es.vec', encoding='ISO-8859-1') as f:
        print("read embedding...")
        for line in f:
            values = line.split(" ")
            word = values[0]
            #print(word)
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
    print("vocabulary size = " + str(len(tokenizer.word_index)))
    print("embedding size = " + str(len(embeddings_index)))
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
    unk_dict = {}
    vocab = len(tokenizer.word_index)
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        elif word in unk_dict:
            embedding_matrix[i] = unk_dict[word]
        else:
            # random init, see https://github.com/bwallace/CNN-for-text-classification/blob/master/CNN_text.py
            unk_embed = np.random.random(300) * -2 + 1
            unk_dict[word] = unk_embed
            embedding_matrix[i] = unk_dict[word]
    return vocab, embedding_matrix
    
def RNN(X,X2):
    vocab1, embedding_matrix1 = buildEmbedding1(X)
    vocab2, embedding_matrix2 = buildEmbedding2(X2)
    en_inputs = Input(name='inputs', shape=(None,)) # se supone que estos son los modelos de fasttext
    es_inputs = Input(name='inputs2', shape=(None,)) # se supone que estos son los modelos de fasttext
    en_embedding = Embedding(vocab1+1, 300, input_length=100, weights=[embedding_matrix1])(en_inputs)
    es_embedding = Embedding(vocab2+1, 300, input_length=100, weights=[embedding_matrix2])(es_inputs)
    lstm1 = LSTM(16)(en_embedding)
    lstm2 = LSTM(16)(es_embedding)
    dense1 = Dense(4,name='FC1')(lstm1)
    dense1 = Activation('relu')(dense1)
    dense1 = Dropout(0.4)(dense1)
    dense2 = Dense(4,name='FC2')(lstm2)
    dense2 = Activation('relu')(dense2)
    dense2 = Dropout(0.4)(dense2)  
    concat = concatenate([dense1, dense2], axis=-1)
    layer = Dense(2,name='out_layer')(concat)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=[en_inputs, es_inputs], outputs=layer)
    return model

### Train

In [32]:
def train_model(train_df, checkpoint_path, change_order=False):
  # Changed from manual parsing to pandas loaded Dataframe
  dataTrain, dataLabel = (train_df['text_translated'].to_list(), train_df['HS'].to_list()) if not change_order else (train_df['text'].to_list(), train_df['HS'].to_list()) # english
  dataTrain2, dataLabel2 = (train_df['text'].to_list(), train_df['HS'].to_list()) if not change_order else (train_df['text_translated'].to_list(), train_df['HS'].to_list()) # spanish

  Y_train = pd.get_dummies(dataLabel)

  max_len = 100
  max_words = 15000
  # We generate a tokenizer for our english text
  tok1 = Tokenizer(num_words=max_words)
  tok1.fit_on_texts(dataTrain)
  vocab1 = len(tok1.word_index)
  # We generate a tokenizer our spanish text
  tok2 = Tokenizer(num_words=max_words)
  tok2.fit_on_texts(dataTrain2)
  vocab2 = len(tok2.word_index)
  # Tokenize and Add padding to our training sequences
  sequences = tok1.texts_to_sequences(dataTrain)
  sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
  # Tokenize and Add padding to our training sequences
  sequences2 = tok2.texts_to_sequences(dataTrain2)
  sequences_matrix2 = sequence.pad_sequences(sequences2, maxlen=max_len)
  # We concatenate training inputs, remember that left is english input and right is spanish input
  allxtrain = [sequences_matrix, sequences_matrix2]
  # Instantiate our model by addin our tokens
  model = RNN(dataTrain, dataTrain2)
  # model.summary()
  model.compile(loss='mse',optimizer=RMSprop(), metrics=[
                                                        tf.keras.metrics.BinaryAccuracy(),
                                                        tf.keras.metrics.Precision(),
                                                        tf.keras.metrics.Recall(),
                                                        tf.keras.metrics.AUC()])
  # Checkpoint callback
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1)
  # Training
  model.fit(allxtrain, Y_train, batch_size=16, epochs=9,
            validation_split=0.2,)
            #callbacks=[EarlyStopping(monitor='val_loss',
            #                         patience=2,
            #                         verbose=0,
            #                         min_delta=0.0001),
            #           cp_callback])
  del model

def test_model(train_df, test_df, checkpoint_path, change_order_train=False, change_order_test=False, save_path_=None):
  # Changed from manual parsing to pandas loaded Dataframe
  dataTrain = train_df['text_translated'].to_list() if not change_order_train else train_df['text'].to_list() # english
  dataTrain2 = train_df['text'].to_list() if not change_order_train else train_df['text_translated'].to_list() # spanish

  dataTest, labelTest = (test_df['text'].to_list(), test_df['HS'].to_list()) if not change_order_test else (test_df['text_translated'].to_list(), test_df['HS'].to_list()) # english
  dataTest2, labelTest2 = (test_df['text_translated'].to_list(), test_df['HS'].to_list()) if not change_order_test else (test_df['text'].to_list(), test_df['HS'].to_list()) # spanish

  Y_test = pd.get_dummies(labelTest)

  max_len = 100
  max_words = 15000
  # We generate a tokenizer for our english text
  tok1 = Tokenizer(num_words=max_words)
  tok1.fit_on_texts(dataTrain)
  vocab1 = len(tok1.word_index)
  # We generate a tokenizer our spanish text
  tok2 = Tokenizer(num_words=max_words)
  tok2.fit_on_texts(dataTrain2)
  vocab2 = len(tok2.word_index)
  # Tokenize and Add padding to our testing sequences
  test_sequences = tok1.texts_to_sequences(dataTest)
  test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
  # Tokenize and Add padding to our testing sequences
  test_sequences2 = tok2.texts_to_sequences(dataTest2)
  test_sequences_matrix2 = sequence.pad_sequences(test_sequences2, maxlen=max_len)

  allxtest = [test_sequences_matrix, test_sequences_matrix2]
  model = tf.keras.models.load_model(checkpoint_path)

  # Hatecheck
  if save_path_:
    test_df_ = test_df.copy()
    raw_predicts = model.predict(allxtest)
    test_df_['HS_predict'] = np.around(raw_predicts[:,1])
    test_df_.to_csv(save_path + "/predictions/" + save_path_)

  return model.evaluate(allxtest, Y_test)


In [29]:
# We create our Full spanish and english datasets
data_hateval_es_full = pd.concat([data_hateval_es_train, data_hateval_es_test])
data_hateval_en_full = pd.concat([data_hateval_en_train, data_hateval_en_test])
data_ours_full = pd.concat([data_train, data_test])

# CL1 + CL2
keep_cols = ['text', 'HS', 'text_translated']
drop_columns_ours = [col for col in data_ours_full.columns.tolist() if col not in keep_cols]
drop_columns_arango = [col for col in data_arango_train.columns.tolist() if col not in keep_cols]

data_chilean = pd.concat([data_ours_full.drop(columns=drop_columns_ours),
           data_arango_train.drop(columns=drop_columns_arango)]).reset_index(drop=True)

### Experiments

The experiments consists in using the full set for training and testing in another language full set

*  Training in spanish and testing in english
*  Training in spanish and testing in chilean arango
*  Training in spanish and testing in chilean ours
*  Training in english and testing in spanish
*  Training in english and testing in spanish




In [30]:
# Checkpoint directory
checkpoint_paths = [
    save_path + 'models/ACL19/es/hateval2019_es.h5',
    save_path + 'models/ACL19/en/hateval2019_en.h5',
    save_path + 'models/ACL19/cl1/hateval2019_cl1.h5',
    save_path + 'models/ACL19/cl2/hateval2019_cl2.h5',
    save_path + 'models/ACL19/cl2/hateval2019_cl1+cl2.h5'
    ]

train_dfs = [
    data_hateval_es_full,
    data_hateval_en_full,
    data_arango_train,
    data_ours_full,
    data_chilean
    ]

change_orders = [
    False,
    True,
    False,
    False,
    False
    ]

train = False
if train:
  train_model(train_dfs[-1], checkpoint_paths[-1], change_orders[-1])
  #for train_data, checkpoint_path, change_order in tqdm(zip(train_dfs[-1], checkpoint_paths[-1], change_orders[-1]), total=len(train_dfs[-1])):
    #train_model(train_data, checkpoint_path, change_order)

In [33]:
es_tests = [
    (data_hateval_en_full, False, True, 'es_en', None),
    (data_arango_train, False, False, 'es_cl1', None),
    (data_ours_full, False, False, 'es_cl2', 'ACL19_predictions_es_cl2.csv'),
    (data_hatecheck, False, False, 'es_hate', 'ACL19_predictions_hatecheck_es_es.csv')
    ]

en_tests = [
    (data_hateval_es_full, True, False, 'en_es', None),
    (data_arango_train, False, False, 'en_cl1', None),
    (data_ours_full, True, False, 'en_cl2', 'ACL19_predictions_en_cl2.csv'),
    (data_hatecheck, True, False, 'en_hate', 'ACL19_predictions_hatecheck_en_es.csv')
    ]

cl1_tests = [
    (data_hateval_es_full, False, False, 'cl1_es', None),
    (data_hateval_en_full, False, True, 'cl1_en', None),
    (data_ours_full, False, False, 'cl1_cl2', 'ACL19_predictions_cl1_cl2.csv'),
    (data_hatecheck, False, False, 'cl1_hate', 'ACL19_predictions_hatecheck_cl1_es.csv')
    ]

cl2_tests = [
    (data_hateval_es_full, False, False, 'cl2_es', None),
    (data_hateval_en_full, False, True, 'cl2_en', None),
    (data_arango_train, False, False, 'cl2_cl1', None),
    (data_hatecheck, False, False, 'cl2_hate', 'ACL19_predictions_hatecheck_cl2_es.csv')
    ]

cl1_cl2_tests = [
    (data_hateval_es_full, False, False, 'clT_es', None),
    (data_hateval_en_full, False, True, 'clT_en', None),
    (data_hatecheck, False, False, 'clT_hate', 'ACL19_predictions_hatecheck_clT_es.csv')
    ]


experiments = [
    es_tests,
    en_tests,
    cl1_tests,
    cl2_tests,
    cl1_cl2_tests
    ]

results_dict = {}

for train_data, experiment, checkpoint_path in tqdm(zip(train_dfs, experiments, checkpoint_paths), total=len(experiments)):
  for test_data, change_order_train, change_order_test, exp_name, save_ in experiment:
    print(exp_name)
    if 'hate' not in exp_name:
      results_dict[exp_name] = test_model(train_data, test_data, checkpoint_path, change_order_train=change_order_train, change_order_test=change_order_test, save_path_=save_)
    else:
      test_model(train_data, test_data, checkpoint_path, change_order_train=change_order_train, change_order_test=change_order_test, save_path_=save_)

  0%|          | 0/5 [00:00<?, ?it/s]

es_en
375/375 [==============================] - 3s 7ms/step - loss: 0.2863 - binary_accuracy: 0.5688 - precision_16: 0.5676 - recall_16: 0.5770 - auc_16: 0.6019
es_cl1
308/308 [==============================] - 5s 9ms/step - loss: 0.1955 - binary_accuracy: 0.6360 - precision_16: 0.6326 - recall_16: 0.6486 - auc_16: 0.7603
es_cl2
143/143 [==============================] - 3s 7ms/step - loss: 0.3186 - binary_accuracy: 0.5095 - precision_16: 0.5093 - recall_16: 0.5197 - auc_16: 0.5229
es_hate
118/118 [==============================] - 2s 7ms/step - loss: 0.2873 - binary_accuracy: 0.5286 - precision_16: 0.5277 - recall_16: 0.5437 - auc_16: 0.4629
en_es
191/191 [==============================] - 3s 11ms/step - loss: 0.2980 - binary_accuracy: 0.5899 - precision_17: 0.5908 - recall_17: 0.5851 - auc_17: 0.6251
en_cl1
308/308 [==============================] - 3s 7ms/step - loss: 0.2314 - binary_accuracy: 0.6827 - precision_17: 0.6855 - recall_17: 0.6752 - auc_17: 0.7205
en_cl2
143/143 [======

In [34]:
results_df = pd.DataFrame.from_dict(results_dict, orient='index')
results_df.rename(columns = {0:'Loss', 1:'Binary Accuracy', 2:'Precision', 3:'Recall', 4:'AUC'}, inplace = True)
results_df['F1-score'] = 2 * results_df['Precision'] * results_df['Recall'] / (results_df['Precision'] + results_df['Recall'])
results_df = results_df[['Loss', 'Binary Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC']]
display(results_df)

results_df.to_csv(save_path + 'ACL19_evaluation_results.csv')
print(results_df.to_latex())


,Loss,Binary Accuracy,Precision,Recall,F1-score,AUC
es_en,0.286258,0.568750,0.567634,0.577000,0.572279,0.601915
es_cl1,0.195452,0.635957,0.632613,0.648566,0.640490,0.760301
es_cl2,0.318572,0.509457,0.509267,0.519683,0.514423,0.522938
en_es,0.297965,0.589918,0.590796,0.585082,0.587925,0.625060
en_cl1,0.231389,0.682733,0.685525,0.675208,0.680328,0.720489
en_cl2,0.337133,0.539587,0.539982,0.534638,0.537297,0.538476
cl1_es,0.368689,0.557377,0.557947,0.552459,0.555189,0.591628
cl1_en,0.367340,0.570042,0.570660,0.565667,0.568152,0.628883
cl1_cl2,0.393396,0.528150,0.528444,0.522982,0.525699,0.531031
cl2_es,0.311464,0.591148,0.590495,0.594754,0.592617,0.638031


\begin{tabular}{lrrrrrr}
\toprule
{} &      Loss &  Binary Accuracy &  Precision &    Recall &  F1-score &       AUC \\
\midrule
es\_en   &  0.286258 &         0.568750 &   0.567634 &  0.577000 &  0.572279 &  0.601915 \\
es\_cl1  &  0.195452 &         0.635957 &   0.632613 &  0.648566 &  0.640490 &  0.760301 \\
es\_cl2  &  0.318572 &         0.509457 &   0.509267 &  0.519683 &  0.514423 &  0.522938 \\
en\_es   &  0.297965 &         0.589918 &   0.590796 &  0.585082 &  0.587925 &  0.625060 \\
en\_cl1  &  0.231389 &         0.682733 &   0.685525 &  0.675208 &  0.680328 &  0.720489 \\
en\_cl2  &  0.337133 &         0.539587 &   0.539982 &  0.534638 &  0.537297 &  0.538476 \\
cl1\_es  &  0.368689 &         0.557377 &   0.557947 &  0.552459 &  0.555189 &  0.591628 \\
cl1\_en  &  0.367340 &         0.570042 &   0.570660 &  0.565667 &  0.568152 &  0.628883 \\
cl1\_cl2 &  0.393396 &         0.528150 &   0.528444 &  0.522982 &  0.525699 &  0.531031 \\
cl2\_es  &  0.311464 &         0.591148 &  

In [ ]:
from scipy.stats import pmean, hmean, gmean

bias_files =  ['ACL19_predictions_hatecheck_es_es.csv',
               'ACL19_predictions_hatecheck_en_es.csv',
               'ACL19_predictions_hatecheck_cl1_es.csv',
               'ACL19_predictions_hatecheck_cl2_es.csv',
               'ACL19_predictions_hatecheck_clT_es.csv']


# Define Evaluation metrics
def compute_metrics_II(p):
    pred, labels = p
    auc = roc_auc_score(y_true=labels, y_score=pred)

    return auc

def compute_metrics(p):
    pred, labels = p

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    auc = roc_auc_score(y_true=labels, y_score=pred)

    return {"accuracy": accuracy, "precision": precision,
            "recall": recall, "f1": f1, "AUC": auc}

def metrics(files):
  results_dict = dict()
  for file_name in files:
    df = pd.read_csv(save_path + "/predictions/" + file_name)
    key_lang = file_name.split('_')[2]
    lang = file_name.split('_')[3]
    l = df['HS'].to_numpy()
    p = df['HS_predict'].to_numpy()
    results_dict[lang + '_' + key_lang] = compute_metrics((p, l))
  results_df = pd.DataFrame.from_dict(results_dict, orient='index')
  display(results_df)
  print(results_df.to_latex())


def bias_metrics(bias_files):
  results_dict = dict()
  for file_ in bias_files:
    df = pd.read_csv(save_path + "/predictions/" + file_)
    key_lang = file_.split('_')[2]
    lang = file_.split('_')[3].split('.')[0]
    for col in df['target_ident'].unique()[:-1]:
      dicc = dict()
      # subgroup AUC
      df_ = df[df['target_ident'] == col]
      l = df_['HS'].to_numpy()
      p = df_['HS_predict'].to_numpy()
      dicc["Subgroup AUC"] = compute_metrics_II((p, l))
      # BPSN AUC
      df_ = df[(df['target_ident'] == col) & (df['HS'] == 0) | (df['target_ident'] != col) & (df['HS'] == 1)]
      l = df_['HS'].to_numpy()
      p = df_['HS_predict'].to_numpy()
      dicc['BPSN AUC'] = compute_metrics_II((p, l))
      # BNSP AUC
      df_ = df[(df['target_ident'] == col) & (df['HS'] == 1) | (df['target_ident'] != col) & (df['HS'] == 0)]
      l = df_['HS'].to_numpy()
      p = df_['HS_predict'].to_numpy()
      dicc['BNSP AUC'] = compute_metrics_II((p, l))
      results_dict[ key_lang + "_" + lang + '_' + col] = dicc
  
  display(pd.DataFrame.from_dict(results_dict, orient='index'))
  print(pd.DataFrame.from_dict(results_dict, orient='index').to_latex())
  dict_results = dict()
  # GMB AUC
  for like in ['es_', 'en_', 'cl1_', 'cl2_', 'clT_']:
    df = pd.DataFrame.from_dict(results_dict, orient='index').filter(like=like, axis=0)
    dict_temp = dict()
    for col in df.columns.tolist():
      dict_temp['GMB ' + col] = pmean(df[col].tolist(), -5)
    dict_results[like.strip('_')] = dict_temp
  display(pd.DataFrame.from_dict(dict_results, orient='index'))
  print(pd.DataFrame.from_dict(dict_results, orient='index').to_latex())


bias_metrics(bias_files)
metrics(bias_files)

,Subgroup AUC,BPSN AUC,BNSP AUC
hatecheck_es_black people,0.465005,0.483084,0.452336
hatecheck_es_gay people,0.486269,0.474703,0.478238
hatecheck_es_trans people,0.501103,0.486551,0.479263
hatecheck_es_indigenous people,0.507635,0.509501,0.462781
hatecheck_es_jews,0.476461,0.497943,0.447882
hatecheck_es_disabled people,0.488704,0.493554,0.462261
hatecheck_es_women,0.514441,0.498747,0.478792
hatecheck_en_black people,0.504094,0.510443,0.491063
hatecheck_en_gay people,0.487122,0.495147,0.491469
hatecheck_en_trans people,0.479217,0.491719,0.488166


\begin{tabular}{lrrr}
\toprule
{} &  Subgroup AUC &  BPSN AUC &  BNSP AUC \\
\midrule
hatecheck\_es\_black people       &      0.465005 &  0.483084 &  0.452336 \\
hatecheck\_es\_gay people         &      0.486269 &  0.474703 &  0.478238 \\
hatecheck\_es\_trans people       &      0.501103 &  0.486551 &  0.479263 \\
hatecheck\_es\_indigenous people  &      0.507635 &  0.509501 &  0.462781 \\
hatecheck\_es\_jews               &      0.476461 &  0.497943 &  0.447882 \\
hatecheck\_es\_disabled people    &      0.488704 &  0.493554 &  0.462261 \\
hatecheck\_es\_women              &      0.514441 &  0.498747 &  0.478792 \\
hatecheck\_en\_black people       &      0.504094 &  0.510443 &  0.491063 \\
hatecheck\_en\_gay people         &      0.487122 &  0.495147 &  0.491469 \\
hatecheck\_en\_trans people       &      0.479217 &  0.491719 &  0.488166 \\
hatecheck\_en\_indigenous people  &      0.489078 &  0.485163 &  0.502630 \\
hatecheck\_en\_jews               &      0.499287 &  0.505635 &  0.

,GMB Subgroup AUC,GMB BPSN AUC,GMB BNSP AUC
es,0.489750,0.491316,0.464981
en,0.493121,0.492604,0.497356
cl1,0.497164,0.497148,0.498027
cl2,0.506318,0.503379,0.505383
clT,0.494242,0.478326,0.495006


\begin{tabular}{lrrr}
\toprule
{} &  GMB Subgroup AUC &  GMB BPSN AUC &  GMB BNSP AUC \\
\midrule
es  &          0.489750 &      0.491316 &      0.464981 \\
en  &          0.493121 &      0.492604 &      0.497356 \\
cl1 &          0.497164 &      0.497148 &      0.498027 \\
cl2 &          0.506318 &      0.503379 &      0.505383 \\
clT &          0.494242 &      0.478326 &      0.495006 \\
\bottomrule
\end{tabular}



,accuracy,precision,recall,f1,AUC
es_hatecheck,0.532443,0.682856,0.625000,0.652648,0.469283
en_hatecheck,0.307610,0.672566,0.028875,0.055373,0.497816
cl1_hatecheck,0.296128,0.166667,0.000380,0.000758,0.497944
cl2_hatecheck,0.317223,0.809917,0.037234,0.071195,0.508285
clT_hatecheck,0.306275,0.693182,0.023176,0.044853,0.499459


\begin{tabular}{lrrrrr}
\toprule
{} &  accuracy &  precision &    recall &        f1 &       AUC \\
\midrule
es\_hatecheck  &  0.532443 &   0.682856 &  0.625000 &  0.652648 &  0.469283 \\
en\_hatecheck  &  0.307610 &   0.672566 &  0.028875 &  0.055373 &  0.497816 \\
cl1\_hatecheck &  0.296128 &   0.166667 &  0.000380 &  0.000758 &  0.497944 \\
cl2\_hatecheck &  0.317223 &   0.809917 &  0.037234 &  0.071195 &  0.508285 \\
clT\_hatecheck &  0.306275 &   0.693182 &  0.023176 &  0.044853 &  0.499459 \\
\bottomrule
\end{tabular}



In [ ]:
files = ['ACL19_predictions_hatecheck_es_es.csv',
         'ACL19_predictions_hatecheck_en_es.csv',
         'ACL19_predictions_hatecheck_cl1_es.csv',
         'ACL19_predictions_hatecheck_cl2_es.csv',
         'ACL19_predictions_hatecheck_clT_es.csv']

def hatecheck(files):
  dicc_ = dict()
  dicc__2 = dict()
  for file_ in files:
    dicc = dict()
    df = pd.read_csv(save_path + "predictions/" + file_)
    filters = df['functionality'].unique()
    for filter in filters:
      df_ = df[df['functionality'] == filter]
      dicc[filter] = accuracy_score(df_['HS'].values, df_['HS_predict'].values)
    dicc_2 = dict()
    filters = df['target_ident'].unique()
    for filter in filters:
      df_ = df[df['target_ident'] == filter]
      dicc_2[filter] = f1_score(df_['HS'].values, df_['HS_predict'].values, average='macro')
    key = file_.split('_')[3]
    print(key)
    dicc_[key] = dicc
    dicc__2[key] = dicc_2
  
  df = pd.DataFrame.from_dict(dicc_, orient='index').transpose()
  print(df.columns.tolist())
  df['delta_cl'] = df['cl2'] - df['cl1']
  display(df)
  print(df.to_latex())
  df = pd.DataFrame.from_dict(dicc__2, orient='index').transpose()
  print(df.columns.tolist())
  df['delta_cl'] = df['cl2'] - df['cl1']
  df = df.drop(index=df.index.tolist()[-1])
  display(df)
  print(df.to_latex())

hatecheck(files)

es
en
cl1
cl2
clT
['es', 'en', 'cl1', 'cl2', 'clT']


/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mea

,es,en,cl1,cl2,clT,delta_cl
derog_neg_emote_h,0.792857,0.000000,0.000000,0.021429,0.007143,0.021429
derog_neg_attrib_h,0.392857,0.000000,0.000000,0.035714,0.014286,0.035714
derog_dehum_h,0.150000,0.000000,0.000000,0.085714,0.014286,0.085714
derog_impl_h,0.800000,0.050000,0.000000,0.114286,0.100000,0.114286
threat_dir_h,0.928571,0.035714,0.000000,0.007143,0.064286,0.007143
threat_norm_h,0.972414,0.034483,0.000000,0.000000,0.013793,0.000000
slur_h,0.900000,0.040000,0.000000,0.000000,0.000000,0.000000
profanity_h,0.564286,0.000000,0.000000,0.085714,0.035714,0.085714
profanity_nh,0.050000,0.980000,1.000000,0.970000,1.000000,-0.030000
ref_subs_clause_h,0.448276,0.020690,0.000000,0.034483,0.013793,0.034483


\begin{tabular}{lrrrrrr}
\toprule
{} &        es &        en &       cl1 &       cl2 &       clT &  delta\_cl \\
\midrule
derog\_neg\_emote\_h  &  0.792857 &  0.000000 &  0.000000 &  0.021429 &  0.007143 &  0.021429 \\
derog\_neg\_attrib\_h &  0.392857 &  0.000000 &  0.000000 &  0.035714 &  0.014286 &  0.035714 \\
derog\_dehum\_h      &  0.150000 &  0.000000 &  0.000000 &  0.085714 &  0.014286 &  0.085714 \\
derog\_impl\_h       &  0.800000 &  0.050000 &  0.000000 &  0.114286 &  0.100000 &  0.114286 \\
threat\_dir\_h       &  0.928571 &  0.035714 &  0.000000 &  0.007143 &  0.064286 &  0.007143 \\
threat\_norm\_h      &  0.972414 &  0.034483 &  0.000000 &  0.000000 &  0.013793 &  0.000000 \\
slur\_h             &  0.900000 &  0.040000 &  0.000000 &  0.000000 &  0.000000 &  0.000000 \\
profanity\_h        &  0.564286 &  0.000000 &  0.000000 &  0.085714 &  0.035714 &  0.085714 \\
profanity\_nh       &  0.050000 &  0.980000 &  1.000000 &  0.970000 &  1.000000 & -0.030000 \\
ref\_subs\_clau

,es,en,cl1,cl2,clT,delta_cl
black people,0.456845,0.208892,0.190826,0.227895,0.190826,0.037069
gay people,0.480396,0.201418,0.190210,0.292106,0.297154,0.101896
trans people,0.493080,0.191864,0.189953,0.259971,0.194054,0.070018
indigenous people,0.494089,0.220121,0.191223,0.196874,0.191223,0.005652
jews,0.464682,0.207335,0.190826,0.200705,0.190826,0.009880
disabled people,0.478661,0.210913,0.191223,0.202479,0.191223,0.011257
women,0.505016,0.267253,0.193600,0.197811,0.194888,0.004211


\begin{tabular}{lrrrrrr}
\toprule
{} &        es &        en &       cl1 &       cl2 &       clT &  delta\_cl \\
\midrule
black people      &  0.456845 &  0.208892 &  0.190826 &  0.227895 &  0.190826 &  0.037069 \\
gay people        &  0.480396 &  0.201418 &  0.190210 &  0.292106 &  0.297154 &  0.101896 \\
trans people      &  0.493080 &  0.191864 &  0.189953 &  0.259971 &  0.194054 &  0.070018 \\
indigenous people &  0.494089 &  0.220121 &  0.191223 &  0.196874 &  0.191223 &  0.005652 \\
jews              &  0.464682 &  0.207335 &  0.190826 &  0.200705 &  0.190826 &  0.009880 \\
disabled people   &  0.478661 &  0.210913 &  0.191223 &  0.202479 &  0.191223 &  0.011257 \\
women             &  0.505016 &  0.267253 &  0.193600 &  0.197811 &  0.194888 &  0.004211 \\
\bottomrule
\end{tabular}

